In [1]:
import streamlit as st
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
import openai

In [2]:
key = ''
location = ''
endpoint = ''

openai.api_type = "azure"
openai.api_key = key
openai.api_base = endpoint
openai.api_version = "2022-12-01"
deployment_id=''

filename = "data/ne.txt"
filename_embeddings = "data/ne_embeddings.pkl"

openai.api_key = key

def create_prompt(context,query):
    header = "Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' \n"
    return header + context + "\n\n" + query + "\n"

def generate_answer(prompt):
    response = openai.Completion.create(
    engine=deployment_id,
    prompt=prompt,
    temperature=0,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop = [' END']
    )
    return (response.choices[0].text).strip()

def clean_text(text):
    '''Make text lowercase,remove punctuation
    .'''
    text = str(text).lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    return text


In [3]:

file1 = open(filename, 'r',encoding="utf8")
Lines = file1.readlines()
file1.close()

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

2023-03-25 13:28:29.397 INFO    sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2023-03-25 13:28:30.931 INFO    sentence_transformers.SentenceTransformer: Use pytorch device: cpu


In [5]:
embeddings = model.encode(Lines)

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

In [6]:
df = pd.DataFrame()
df['text'] = Lines
df['embeddings'] = embeddings.tolist()

In [7]:
df.head()

,text,embeddings
0,"From Wikipedia, the free encyclopedia\n","[-0.01346847228705883, -0.0015914966352283955,..."
1,This article is about the North Eastern Region...,"[0.018806032836437225, 0.06984338164329529, 0...."
2,Northeast India\n,"[0.06450401246547699, 0.024881865829229355, -0..."
3,North Eastern Region (NER)\n,"[0.0062939138151705265, 0.07466636598110199, 0..."
4,Northeast india map.png\n,"[0.07192499190568924, 0.060274820774793625, -0..."


In [8]:
pickle.dump(df,open(filename_embeddings,"wb"))

In [9]:
with open(filename_embeddings, 'rb') as f:
    df_embeddings = pickle.load(f)

In [10]:
user_input="What are the geographical features in North East?"

In [11]:
q_new = user_input
q_new = [model.encode(q_new)]
result = cosine_similarity(q_new,df_embeddings["embeddings"].tolist())
result_df = pd.DataFrame(result[0], columns = ['sim'])
df = pd.DataFrame(df_embeddings["text"].tolist(),columns = ["text"])
q = pd.concat([df,result_df],axis = 1)
q = q.sort_values(by="sim",ascending = False)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
q.head()

,text,sim
3,North Eastern Region (NER)\n,0.634095
99,Geography\n,0.605726
69,Formation of North Eastern states\n,0.592888
1,This article is about the North Eastern Region...,0.547714
2,Northeast India\n,0.505196


In [13]:
context= "\n\n".join(q["text"][:50])
    
prompt = create_prompt(context,user_input)
reply = generate_answer(prompt)

In [14]:
reply

'The geographical features in North East include the Eastern Himalaya, the Patkai and the Brahmaputra and the Barak valley plains, mountains and hills by state, the Brahmaputra Valley and Eastern Himalaya, Dzüko Valley (Borders of Nagaland and Manipur), Kangto (shared with China) and Kangchenjunga (shared with Nepal).'